In [1]:
import pandas as pd
import sys

In [3]:
df = pd.read_csv(r"C:\Users\betul\Desktop\archive\spotify_millsongdata.csv")


In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)


,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape


(57650, 4)

In [7]:
df.isnull().sum()


artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)


In [9]:
print(df.columns)


Index(['artist', 'song', 'text'], dtype='object')


In [10]:
df.head(10)


,artist,song,text
0,Bob Marley,There She Goes,There she goes with the pieces from my heart ...
1,Ramones,Out Of Time,You don't know what's going on \r\nYou've bee...
2,Beach Boys,Break Away,"Break break shake away, break breakaway \r\nN..."
3,Willie Nelson,Everything's Beautiful (In It's Own Way),When I look out over a green field of clover ...
4,Dream Theater,In The Presence Of Enemies Part 1,"I saw a bright light, shinning there before \..."
5,Tom Jones,It's Just A Matter Of Time,"Someday, some way, you'll realize that you've ..."
6,Morrissey,Now I Am A Was,Once I was complete and \r\nSafe within your ...
7,Squeeze,F-Hole,I wrote her name on a bar mat \r\nShe had a p...
8,George Strait,I'm Never Gonna Let You Go,Let's turn the lights down soft and low \r\nA...
9,Xscape,How Do You Love Someone?,"[Chorus] \r\nHow do you love someone, \r\nWh..."


In [11]:
df['text'][0]


"There she goes with the pieces from my heart  \r\nThere she goes and now my teardrops start  \r\nAnd ere I go once again  \r\nTru deese loonely eartaches and pain  \r\n  \r\nThat's all remain all remain  \r\nNow I know love is a guessin' game  \r\nShe goes with the pieces from my heart  \r\nThere she goes and now my teardrops start  \r\n  \r\nWho's gonna put back the pieces to my broken heart  \r\nOnce again once again now I know this could be the end  \r\nAnd she gone with the pieces of my heart  \r\nThere she goes and now my teardrops start  \r\n  \r\nAnd here I go once again  \r\nTru deese lonely eartaches and pains  \r\nThat's all remain  \r\nNow I know this could be the end  \r\n  \r\nShe goes wid de pieces from my eart  \r\nThere she goes and now my teardrops start\r\n\r\n"

In [12]:
#df = df.sample(5000)


In [13]:
df.shape


(5000, 3)

In [14]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [20]:
similarity[0]


array([1.        , 0.00255178, 0.00417462, ..., 0.        , 0.        ,
       0.05663876])

In [21]:
df[df['song'] == 'Crying Over You']


,artist,song,text


In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [23]:
print(df['song'].unique())


['There She Goes' 'Out Of Time' 'Break Away' ... 'Deck The Halls'
 'They Hung Him On A Cross' "Hell's Ditch"]


In [24]:


def recommendation(song_name):
    if not df[df['song'] == song_name].empty:
        idx = df[df['song'] == song_name].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
        songs = []
        # Şarkıları listeleme işlemi burada devam eder
    else:
        print(f"'{song_name}' şarkısı veri çerçevesinde bulunamadı.")

recommendation('Crying Over You')

'Crying Over You' şarkısı veri çerçevesinde bulunamadı.


In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))